In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
#transactions table
transactions = pd.read_csv("transactions_unique.csv")

In [ ]:
transaction_count = (
    transactions.groupby(["Store_ID", "SKU_ID"])
      .size()
      .reset_index(name="Transaction_Count")
)


In [ ]:
def build_initiation_inventory(transactions_df, buffer_days=0):
    """
    Step 1:
    Create inventory dataframe with columns:
    Store_ID, SKU_ID, Inventory_Date, Units_Sold
    """

    # Keep only required columns & remove duplicates
    df = transactions_df[
        ["Store_ID", "SKU_ID", "Last_Purchase_Date", "Last_Order_Qty"]
    ].drop_duplicates()

    df["Last_Purchase_Date"] = pd.to_datetime(df["Last_Purchase_Date"])

    rows = []

    for (store, sku), g in tqdm(df.groupby(["Store_ID", "SKU_ID"])):

        # Date range per SKU
        start_date = g["Last_Purchase_Date"].min()
        end_date = g["Last_Purchase_Date"].max() + pd.Timedelta(days=buffer_days)

        all_dates = pd.date_range(start_date, end_date, freq="D")

        # Map transaction dates to quantities
        sales_map = (
            g.groupby("Last_Purchase_Date")["Last_Order_Qty"]
            .sum()
            .to_dict()
        )

        for d in all_dates:
            rows.append([
                store,
                sku,
                d,
                sales_map.get(d, 0)
            ])

    inv_df_1 = pd.DataFrame(
        rows,
        columns=["Store_ID", "SKU_ID", "Inventory_Date", "Units_Sold"]
    )

    return inv_df_1


In [ ]:
def derive_inventory_assumptions(inv_df_1):
    """
    Step 2:
    Derive static inventory assumptions per Store–SKU
    using only inv_df_1
    """

    rows = []

    for (store, sku), g in tqdm(inv_df_1.groupby(["Store_ID", "SKU_ID"])):

        sales = g[g["Units_Sold"] > 0]["Units_Sold"]

        # Safety check
        if len(sales) == 0:
            continue

        typical_sale_qty = int(np.median(sales))
        max_sale_qty = int(sales.max())

        # ---- Assumptions ----
        ROP = typical_sale_qty
        TARGET_STOCK = ROP * 3
        INITIAL_OPENING_STOCK = TARGET_STOCK

        rows.append([
            store,
            sku,
            INITIAL_OPENING_STOCK,
            ROP,
            TARGET_STOCK
        ])

    assumptions_df = pd.DataFrame(
        rows,
        columns=[
            "Store_ID",
            "SKU_ID",
            "Initial_Opening_Stock",
            "Reorder_Point",
            "Target_Stock"
        ]
    )

    return assumptions_df


In [ ]:
import pandas as pd

def build_inventory_final(inv_df_1, assumptions_df):
    """
    Step 3:
    Expand inv_df_1 into full inventory simulation
    using assumptions_df
    """

    # Sort for correct day-by-day processing
    inv_df_1 = inv_df_1.sort_values(
        ["Store_ID", "SKU_ID", "Inventory_Date"]
    )

    rows = []

    for (store, sku), g in tqdm(inv_df_1.groupby(["Store_ID", "SKU_ID"])):

        g = g.sort_values("Inventory_Date")

        # ---- Fetch assumptions ----
        a = assumptions_df[
            (assumptions_df["Store_ID"] == store) &
            (assumptions_df["SKU_ID"] == sku)
        ].iloc[0]

        opening_stock = a["Initial_Opening_Stock"]
        ROP = a["Reorder_Point"]
        TARGET_STOCK = a["Target_Stock"]

        for _, row in g.iterrows():

            units_sold = row["Units_Sold"]

            closing_stock = max(opening_stock - units_sold, 0)

            rows.append([
                store,
                sku,
                row["Inventory_Date"],
                units_sold,
                opening_stock,
                closing_stock,
                ROP
            ])

            # ---- Replenishment logic ----
            if closing_stock <= ROP:
                opening_stock = TARGET_STOCK
            else:
                opening_stock = closing_stock

    inventory_final = pd.DataFrame(
        rows,
        columns=[
            "Store_ID",
            "SKU_ID",
            "Inventory_Date",
            "Units_Sold",
            "Estimated_Opening_Stock",
            "Estimated_Closing_Stock",
            "Reorder_Level"
        ]
    )

    return inventory_final


In [ ]:
def derive_inventory(transactions):
  initiate_inventory=build_initiation_inventory(transactions)
  assumptions=derive_inventory_assumptions(initiate_inventory)
  inventory=build_inventory_final(initiate_inventory, assumptions)
  return inventory, assumptions, initiate_inventory


In [ ]:
inventory, assumptions, initiate_inventory=derive_inventory(transactions.drop_duplicates())

100%|██████████| 2051/2051 [00:18<00:00, 110.56it/s]


In [ ]:
#final derived table. -inventory table
inventory

,Store_ID,SKU_ID,Inventory_Date,Units_Sold,Estimated_Opening_Stock,Estimated_Closing_Stock,Reorder_Level
0,S001,SKU_1,2023-08-06,18,48,30,16
1,S001,SKU_1,2023-08-07,0,30,30,16
2,S001,SKU_1,2023-08-08,0,30,30,16
3,S001,SKU_1,2023-08-09,0,30,30,16
4,S001,SKU_1,2023-08-10,0,30,30,16
...,...,...,...,...,...,...,...
285355,S050,SKU_9,2023-12-26,0,55,55,27
285356,S050,SKU_9,2023-12-27,0,55,55,27
285357,S050,SKU_9,2023-12-28,0,55,55,27
285358,S050,SKU_9,2023-12-29,0,55,55,27


In [ ]:
#checking values for a particular SKU of a store in transactions table
transactions[(transactions['Store_ID']=='S007')& (transactions['SKU_ID']=='SKU_36')]

,Store_ID,SKU_ID,Last_Purchase_Date,Avg_Order_Qty,Last_Order_Qty,Category
363,S007,SKU_36,2023-12-21,34,24,Home Care
932,S007,SKU_36,2023-08-16,29,29,Home Care
2464,S007,SKU_36,2023-12-02,29,21,Home Care
3228,S007,SKU_36,2024-02-12,31,22,Home Care
5643,S007,SKU_36,2024-03-13,30,27,Home Care
5685,S007,SKU_36,2024-06-01,37,31,Home Care
6563,S007,SKU_36,2023-11-15,31,22,Home Care
6660,S007,SKU_36,2023-07-20,29,27,Home Care
6770,S007,SKU_36,2024-04-13,34,23,Home Care
7108,S007,SKU_36,2024-06-06,30,30,Home Care


In [ ]:
#checking corresponding values for a particular SKU of a store in inventory table

inventory[(inventory['Store_ID']=='S007')& (inventory['SKU_ID']=='SKU_36')]

,Store_ID,SKU_ID,Inventory_Date,Units_Sold,Estimated_Opening_Stock,Estimated_Closing_Stock,Reorder_Level
40194,S007,SKU_36,2023-07-20,27,72,45,24
40195,S007,SKU_36,2023-07-21,0,45,45,24
40196,S007,SKU_36,2023-07-22,0,45,45,24
40197,S007,SKU_36,2023-07-23,0,45,45,24
40198,S007,SKU_36,2023-07-24,0,45,45,24
...,...,...,...,...,...,...,...
40512,S007,SKU_36,2024-06-02,0,72,72,24
40513,S007,SKU_36,2024-06-03,0,72,72,24
40514,S007,SKU_36,2024-06-04,0,72,72,24
40515,S007,SKU_36,2024-06-05,0,72,72,24
